In [1]:
#### Virtual assistants and  accessing data

In [2]:
## SQLite with Python

In [3]:
import sqlite3
conn = sqlite3.connect('hotels.db')
c = conn.cursor()
c.execute("DROP TABLE hotels")
c.execute("CREATE TABLE IF NOT EXISTS hotels(name text, price text, location text, stars int)")
c.execute("INSERT INTO hotels(name, price, location, stars) VALUES('Hotel for Dogs', 'mid', 'east', 3)")
c.execute("INSERT INTO hotels(name, price, location, stars) VALUES('Hotel California', 'mid', 'north', 3)")
c.execute("INSERT INTO hotels(name, price, location, stars) VALUES('Grand Hotel', 'hi', 'south', 5)")
c.execute("INSERT INTO hotels(name, price, location, stars) VALUES('Cozy Cottage', 'lo', 'south', 2)")
c.execute("INSERT INTO hotels(name, price, location, stars) VALUES('Bens BnB', 'hi', 'north', 4)")
c.execute("INSERT INTO hotels(name, price, location, stars) VALUES('The Grand', 'hi', 'west', 5)")
c.execute("INSERT INTO hotels(name, price, location, stars) VALUES('Central Rooms', 'mid', 'center', 3)")
c.execute("commit")

In [4]:
c.execute("SELECT * FROM hotels")

In [5]:
print(c.fetchall())

[('Hotel for Dogs', 'mid', 'east', 3), ('Hotel California', 'mid', 'north', 3), ('Grand Hotel', 'hi', 'south', 5), ('Cozy Cottage', 'lo', 'south', 2), ('Bens BnB', 'hi', 'north', 4), ('The Grand', 'hi', 'west', 5), ('Central Rooms', 'mid', 'center', 3)]


In [6]:
## Exploring a DB with  natural language

In [7]:
# Parameters from text

In [ ]:
# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# Create a trainer that uses this config
trainer = Trainer(config.load("config_spacy.yml"))

# Load the training data
training_data = load_data('demo-rasa.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

In [9]:
message = "a cheap hotel in the north"

In [10]:
data = interpreter.parse(message)
data

{'intent': {'name': 'hotel_search', 'confidence': 0.362941684880824},
 'entities': [{'start': 2,
   'end': 7,
   'value': 'lo',
   'entity': 'price',
   'confidence': 0.7868873766314981,
   'extractor': 'CRFEntityExtractor',
   'processors': ['EntitySynonymMapper']},
  {'start': 21,
   'end': 26,
   'value': 'north',
   'entity': 'location',
   'confidence': 0.5880084511285073,
   'extractor': 'CRFEntityExtractor'}],
 'intent_ranking': [{'name': 'hotel_search', 'confidence': 0.362941684880824},
  {'name': 'restaurant_search', 'confidence': 0.23641221266967594},
  {'name': 'goodbye', 'confidence': 0.1237479987258443},
  {'name': 'affirm', 'confidence': 0.12000665828034478},
  {'name': 'greet', 'confidence': 0.08842061207561007},
  {'name': 'location', 'confidence': 0.06847083336770085}],
 'text': 'a cheap hotel in the north'}

In [11]:
params = {}
for ent in data["entities"]:
    params[ent["entity"]] = ent["value"]
params

{'price': 'lo', 'location': 'north'}

In [12]:
## Creating a query from parameters

In [13]:
query = "select name FROM hotels"

In [14]:
filters = ["{}=?".format(k) for k in params.keys()]

In [15]:
filters

['price=?', 'location=?']

In [16]:
conditions = " and ".join(filters)

In [17]:
conditions

'price=? and location=?'

In [18]:
final_q = " WHERE ".join([query, conditions])

In [19]:
final_q

'select name FROM hotels WHERE price=? and location=?'

In [20]:
## Responses

In [21]:
responses = [
    "I'm sorry :( I couldn't find anything like that",  
    "what about {}?",
    "{} is one option, but I know others too :)"
]

In [22]:
results = c.fetchall() 

In [23]:
len(results)

0

In [24]:
index = min(len(results), len(responses)-1)  

In [25]:
responses[index]

"I'm sorry :( I couldn't find anything like that"

In [26]:
## Basic Memory

In [27]:
def respond(message, params):
    # update params with entities in message   
    # run query
    # pick response
    return response, params

In [28]:
# initialise params. params has memory
params = {}

In [ ]:
# message comes in
response, params = respond(message, params)

In [30]:
## Catching negations
import spacy
nlp = spacy.load("en_core_web_md")

In [31]:
doc = nlp('not sushi, maybe pizza?')

In [32]:
indices = [1, 4]

In [33]:
ents, negated_ents = [], []

In [34]:
start = 0
for i in indices:
    phrase = "{}".format(doc[start:i])
    if "not" in phrase or "n't" in phrase: 
        negated_ents.append(doc[i])
        print(negated_ents) #有问题！
    else:
        ents.append(doc[i])
        print(ents)
    start = i

print(ents)
print(negated_ents)

[sushi]
[pizza]
[pizza]
[sushi]


In [35]:
query = 'SELECT * FROM hotels'
filters = ['price=?']
print(filters)
query += " WHERE " + " and ".join(filters)
print(query)

['price=?']
SELECT * FROM hotels WHERE price=?


In [36]:
x = [1,2,3]
x.append(5)
print(x)

[1, 2, 3, 5]
